<a href="https://colab.research.google.com/github/Sandeepkumaramgothu/ASE_practice/blob/main/Spect_2_Gun_Other.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
!pip install --upgrade keras
!pip install --upgrade keras tensorflow
import os
import math
import numpy as np
import librosa
import json
import pandas as pd
from termcolor import colored
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
!pip install tensorboard
!pip install visualkeras
import visualkeras
from keras.utils import plot_model
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.svm import SVC
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from IPython.display import Audio
from scipy.io import wavfile
import scipy
import soundfile as sf
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
import random



In [16]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.metrics import classification_report, confusion_matrix

def create_mel_spectrogram(file_path, save_path, n_mels=128, n_fft=2048, hop_length=512, sr=16000):
    """ This function generates a mel spectrogram from a sound file and saves it as an image. """
    y, sr = librosa.load(file_path, sr=sr)

    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
    S_DB = librosa.amplitude_to_db(S, ref=np.max)

    plt.figure(figsize=(10, 4))
    librosa.display.specshow(S_DB, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel spectrogram')
    plt.tight_layout()

    plt.savefig(save_path)
    #plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
    print(f'Mel spectrogram saved as {save_path}\n')
    plt.close()

def prepare_dataset(sound_dir, image_dir):
    """ Processes each category in train and test directories to create spectrogram images. """
    for set_type in ['Train', 'Test']:
        set_sound_dir = os.path.join(sound_dir, set_type)
        set_image_dir = os.path.join(image_dir, set_type)
        categories = os.listdir(set_sound_dir)
        for category in categories:
            filecounter=0
            print(f'Processing {category} category... in {set_type} ')
            cat_sound_dir = os.path.join(set_sound_dir, category)
            cat_image_dir = os.path.join(set_image_dir, category)
            os.makedirs(cat_image_dir, exist_ok=True)
            sound_files = [f for f in os.listdir(cat_sound_dir) if f.endswith('.wav') or f.endswith('.mp3')]
            for sound_file in sound_files:
                file_path = os.path.join(cat_sound_dir, sound_file)
                print(f'Processing {file_path}')
                image_path = os.path.join(cat_image_dir, sound_file.replace('.wav', '.png'))
                create_mel_spectrogram(file_path, image_path)
                filecounter+=1
            print(f"Processed {category} with {filecounter} files \n\n")

base_dir = '/content/drive/MyDrive/DataSet/Dataset_Version_2'
sound_dir = os.path.join(base_dir, 'Sound')
image_dir = os.path.join(base_dir, 'Images')

prepare_dataset(sound_dir, image_dir)



Processing AK-12 category... in Train 
Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound/Train/AK-12/3 (1).wav
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images/Train/AK-12/3 (1).png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound/Train/AK-12/3 (30).wav
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images/Train/AK-12/3 (30).png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound/Train/AK-12/3 (29).wav
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images/Train/AK-12/3 (29).png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound/Train/AK-12/3 (26).wav
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images/Train/AK-12/3 (26).png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound/Train/AK-12/3 (28).wav
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images/Train/AK-12/3 (28

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(image_dir, 'Train'),
    target_size=(128, 256),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    os.path.join(image_dir, 'Test'),
    target_size=(128, 256),
    batch_size=32,
    class_mode='binary'
)



In [13]:


model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 256, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_generator, epochs=150, validation_data=test_generator)




/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/150


/usr/local/lib/python3.11/dist-packages/keras/src/ops/nn.py:907: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/losses/losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.3116 - loss: 0.0000e+00 - val_accuracy: 0.3333 - val_loss: 0.0000e+00
Epoch 2/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 370ms/step - accuracy: 0.3019 - loss: 0.0000e+00 - val_accuracy: 0.3333 - val_loss: 0.0000e+00
Epoch 3/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 390ms/step - accuracy: 0.3639 - loss: 0.0000e+00 - val_accuracy: 0.3333 - val_loss: 0.0000e+00
Epoch 4/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 391ms/step - accuracy: 0.3212 - loss: 0.0000e+00 - val_accuracy: 0.3333 - val_loss: 0.0000e+00
Epoch 5/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 584ms/step - accuracy: 0.3216 - loss: 0.0000e+00 - val_accuracy: 0.3333 - val_loss: 0.0000e+00
Epoch 6/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 379ms/step - accuracy: 0.3082 - loss: 0.0000e+00 - val_accuracy: 0.3333 - val_loss: 0.0000e+00
Epoch 7/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 384ms/step - accuracy: 0.3737 - loss: 0.0000e+00 - val_accuracy: 0.3333 - val_loss: 0.0000e+00
Epoch 8/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 362ms/step - accuracy: 0.3709

In [14]:
results = model.evaluate(test_generator)
print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")

predictions = model.predict(test_generator)

y_pred = (predictions > 0.5).astype(int).flatten()
y_true = test_generator.classes

print("predicted values : ",y_pred)
print("True values      : ",y_true)


print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))
print("\nClassification Report:")
print(classification_report(y_true, y_pred))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step - accuracy: 0.3333 - loss: 0.0000e+00
Test Loss: 0.0
Test Accuracy: 0.3333333432674408


/usr/local/lib/python3.11/dist-packages/keras/src/ops/nn.py:907: UserWarning: You are using a softmax over axis -1 of a tensor of shape (30, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 528ms/step
predicted values :  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
True values      :  [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2]

Confusion Matrix:
[[ 0 10  0]
 [ 0 10  0]
 [ 0 10  0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.33      1.00      0.50        10
           2       0.00      0.00      0.00        10

    accuracy                           0.33        30
   macro avg       0.11      0.33      0.17        30
weighted avg       0.11      0.33      0.17        30



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
